## **CONGRESSO EM TEMAS** DASH

---


In [3]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib inline

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode(connected=True)

# con = create_engine(server[db_cts])
plotly.__version__

'2.5.0'

---

## **DATA PREPARATION**

### **Propositions**

#### Get propositions infos

From *proposicoes_id* table. <br>
The query request only the propositions which type is PL, PEC, PLV, MPV, PLV or PDC.

In [4]:
columns = ['idproposicao','nomeproposicao','tema','tipoproposicao',
           'dataapresentacao','ementa','indexacao','situacao',
           'idecadastro', 'autor','ufautor','regimetramitacao',
           'ultimodespacho']

query_propositions = """
SELECT DISTINCT
  {}
FROM
  c_camdep.camdep_proposicoes_id t1,
  (SELECT DISTINCT
      idproposicao,
      max(capnum) OVER (PARTITION BY idproposicao) capnum
   FROM
      c_camdep.camdep_proposicoes_id) t2
WHERE
  t1.idproposicao = t2.idproposicao
  AND t1.capnum = t2.capnum
  AND t1.nomeproposicao SIMILAR TO '(PL|PLP|PLV|MPV|PDC|PEC) %%' 
  AND t1.dataapresentacao >= '2003-01-01'
""".format(','.join(['t1.' + column for column in columns]))

query_results_propositions = pd.read_sql_query(query_propositions,con)
print('propositions OK!')

propositions OK!


In [3]:
propositions = deepcopy(query_results_propositions)
print(propositions.info())
propositions.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48978 entries, 0 to 48977
Data columns (total 13 columns):
idproposicao        48978 non-null int64
nomeproposicao      48978 non-null object
tema                48776 non-null object
tipoproposicao      48978 non-null object
dataapresentacao    48978 non-null datetime64[ns]
ementa              48978 non-null object
indexacao           48709 non-null object
situacao            48958 non-null object
idecadastro         36790 non-null float64
autor               48978 non-null object
ufautor             38835 non-null object
regimetramitacao    48978 non-null object
ultimodespacho      47413 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(10)
memory usage: 4.9+ MB
None


,idproposicao,nomeproposicao,tema,tipoproposicao,dataapresentacao,ementa,indexacao,situacao,idecadastro,autor,ufautor,regimetramitacao,ultimodespacho
0,416576,PL 4334/2008,"Indústria, Comércio e Defesa do Consumidor (ut...",Projeto de Lei,2008-11-19,"Acrescenta dispositivos da Lei nº 8.078, de 11...","Alteração, Código de Defesa do Consumidor, fix...",MESA - Arquivada,141384.0,Valadares Filho,SE,Urgência art. 155 RICD,Apense-se à(ao) PL-836/2003. Proposição Sujeit...
1,446624,PL 5848/2009,"Indústria, Comércio e Defesa do Consumidor (ut...",Projeto de Lei,2009-08-25,"Altera a Lei nº 8.078, de 11 de setembro de 19...","Alteração, Código de Defesa do Consumidor, pra...",MESA - Aguardando Deliberação de Recurso,141401.0,Carlos Bezerra,MT,Ordinária,Às Comissões de Defesa do Consumidor e Const...


#### Coleting list of themes

In [4]:
themes = propositions.loc[~propositions.tema.isnull(),'tema']
themes = themes.apply(lambda x: x.replace(' (utilizado até a legislatura 53)',''))\
               .drop_duplicates()\
               .apply(lambda x: x.split(';'))
unique_themes = []
for i in themes:
    for theme in i:
        theme = theme.strip(' ')
        if theme not in unique_themes:
            unique_themes.append(theme)
unique_themes.sort()
unique_themes = pd.DataFrame(unique_themes,columns=['Tema'])
unique_themes.index.name = 'cod_tema'
unique_themes.to_csv('themes.csv')

#### Cataloging propositions by situation

Based on analisys produced by Congresso em Números. [Document link here](https://docs.google.com/document/d/1qqcfJMybpvjKJ63i77YcVbIUzUezkTpnnqbBSJmXMT0/edit?usp=sharing).

In [5]:
propositions['descricao_situacao'] = 'Em tramitação'

classification = {
    'Aprovada': [
        'Tranformada',
        'Transformado em Norma Jurídica',    
        'Aguardando Apreciação pelo Senado Federal',
        'Aguardando Envio ao Senado Federal',
        'Enviada ao Senado Federal',
        'Aguardando Promulgação',
        'Aguardando Apreciação do Veto',
        'Aguardando Sanção',
        'Aguardando Envio ao Executivo',
        'Vetado Totalmente',
        'Originou'    
    ],
    'Arquivada': [
        'Arquivada',
        'Aguardando Despacho de Arquivamento',
        'Aguardando Remessa ao Arquivo',
        'Enviada ao arquivo',
        'Perdeu a eficácia',
        'Transformado em nova proposição'
    ]
}

for situation in tqdm(classification):
    sits = '|'.join(classification[situation]) 
    propositions.loc[propositions['situacao'].str.contains(sits) == True, 'descricao_situacao'] = situation

100%|██████████| 2/2 [00:00<00:00,  8.73it/s]


In [6]:
propositions.head(2)

,idproposicao,nomeproposicao,tema,tipoproposicao,dataapresentacao,ementa,indexacao,situacao,idecadastro,autor,ufautor,regimetramitacao,ultimodespacho,descricao_situacao
0,416576,PL 4334/2008,"Indústria, Comércio e Defesa do Consumidor (ut...",Projeto de Lei,2008-11-19,"Acrescenta dispositivos da Lei nº 8.078, de 11...","Alteração, Código de Defesa do Consumidor, fix...",MESA - Arquivada,141384.0,Valadares Filho,SE,Urgência art. 155 RICD,Apense-se à(ao) PL-836/2003. Proposição Sujeit...,Arquivada
1,446624,PL 5848/2009,"Indústria, Comércio e Defesa do Consumidor (ut...",Projeto de Lei,2009-08-25,"Altera a Lei nº 8.078, de 11 de setembro de 19...","Alteração, Código de Defesa do Consumidor, pra...",MESA - Aguardando Deliberação de Recurso,141401.0,Carlos Bezerra,MT,Ordinária,Às Comissões de Defesa do Consumidor e Const...,Em tramitação


#### Get proceedings of propositions

From *proposicoes_tramitadas_periodo* table. <br>
The query request only the last proceeding of each proposition. 



#### While we can't use *proposicoes_andamento*, let's query table *proposicoes_tramitadas_periodo* 

In [7]:
query_proceedings = """
SELECT DISTINCT
  codproposicao idproposicao,
  max(datatramitacao) OVER (PARTITION BY codproposicao) data_ultima_tramitacao
FROM
  viz_congresso_tramitacao.proposicoes_tramitadas_periodo
WHERE 
  datatramitacao  >= '2003-01-01'
"""

proceedings = pd.read_sql_query(query_proceedings,con)
proceedings.head(2)

,idproposicao,data_ultima_tramitacao
0,12666,2012-07-31
1,12669,2007-04-02


In [8]:
propositions = propositions.merge(proceedings, on='idproposicao', how='left')
print(propositions.info())
propositions.head(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48978 entries, 0 to 48977
Data columns (total 15 columns):
idproposicao              48978 non-null int64
nomeproposicao            48978 non-null object
tema                      48776 non-null object
tipoproposicao            48978 non-null object
dataapresentacao          48978 non-null datetime64[ns]
ementa                    48978 non-null object
indexacao                 48709 non-null object
situacao                  48958 non-null object
idecadastro               36790 non-null float64
autor                     48978 non-null object
ufautor                   38835 non-null object
regimetramitacao          48978 non-null object
ultimodespacho            47413 non-null object
descricao_situacao        48978 non-null object
data_ultima_tramitacao    46734 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(11)
memory usage: 6.0+ MB
None


,idproposicao,nomeproposicao,tema,tipoproposicao,dataapresentacao,ementa,indexacao,situacao,idecadastro,autor,ufautor,regimetramitacao,ultimodespacho,descricao_situacao,data_ultima_tramitacao
0,416576,PL 4334/2008,"Indústria, Comércio e Defesa do Consumidor (ut...",Projeto de Lei,2008-11-19,"Acrescenta dispositivos da Lei nº 8.078, de 11...","Alteração, Código de Defesa do Consumidor, fix...",MESA - Arquivada,141384.0,Valadares Filho,SE,Urgência art. 155 RICD,Apense-se à(ao) PL-836/2003. Proposição Sujeit...,Arquivada,2009-05-19
1,446624,PL 5848/2009,"Indústria, Comércio e Defesa do Consumidor (ut...",Projeto de Lei,2009-08-25,"Altera a Lei nº 8.078, de 11 de setembro de 19...","Alteração, Código de Defesa do Consumidor, pra...",MESA - Aguardando Deliberação de Recurso,141401.0,Carlos Bezerra,MT,Ordinária,Às Comissões de Defesa do Consumidor e Const...,Em tramitação,2011-05-31


There is a small problem: Some propositions aren't in *proposicoes_tramitadas_periodo*. After a simple verification, appear that it's a error from Câmara dos Deputados API.

**We will need to decide what we should do.**

In [9]:
propositions.to_sql('proposicoes',con,schema='viz_painel_temas',if_exists='replace')
#propositions.to_sql('proposicoes',con,schema='viz_painel_temas',if_exists='fail')

<a id=#Congressmen></a>
### **Congressmen**

#### Get congressmen infos

From *camdep_deputatdos_historico* table.

In [10]:
columns = ['idecadastro', 'numlegislatura', 'nomeparlamentar',
           'sexo', 'legendapartidoeleito', 'condicao', 
           'situacaomandato', 'ufeleito']

query_congressmen = """
    SELECT
        {}
    FROM
        c_camdep.camdep_deputados_historico
    
""".format(','.join(columns))

query_results_congressmen = pd.read_sql_query(query_congressmen,con)

In [11]:
congressmen = deepcopy(query_results_congressmen)
print(congressmen.info())
congressmen.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2481 entries, 0 to 2480
Data columns (total 8 columns):
idecadastro             2481 non-null int64
numlegislatura          2481 non-null int64
nomeparlamentar         2481 non-null object
sexo                    2481 non-null object
legendapartidoeleito    2481 non-null object
condicao                2481 non-null object
situacaomandato         2481 non-null object
ufeleito                2481 non-null object
dtypes: int64(2), object(6)
memory usage: 155.1+ KB
None


,idecadastro,numlegislatura,nomeparlamentar,sexo,legendapartidoeleito,condicao,situacaomandato,ufeleito
0,178957,55,ABEL MESQUITA JR.,M,PDT,Titular,Em Exercício,RR
1,141463,53,ABELARDO CAMARINHA,M,PSB,Titular,Em Exercício,SP


#### Get url pictures

From *br_camdep_v2.deputados* table.

In [12]:
columns = ['idecadastro', 'urlfoto']

query_pictures = """
SELECT
    {}
  FROM
    br_camdep_v2.deputados
""".format(','.join(columns))

pictures = pd.read_sql_query(query_pictures,con)
pictures.head(2)

,idecadastro,urlfoto
0,178957,http://www.camara.leg.br/internet/deputado/ban...
1,141463,http://www.camara.leg.br/internet/deputado/ban...


In [13]:
congressmen = congressmen.merge(
    pictures, left_on='idecadastro', right_on='idecadastro', how='left'
).drop_duplicates()
congressmen.head(2)

,idecadastro,numlegislatura,nomeparlamentar,sexo,legendapartidoeleito,condicao,situacaomandato,ufeleito,urlfoto
0,178957,55,ABEL MESQUITA JR.,M,PDT,Titular,Em Exercício,RR,http://www.camara.leg.br/internet/deputado/ban...
1,141463,53,ABELARDO CAMARINHA,M,PSB,Titular,Em Exercício,SP,http://www.camara.leg.br/internet/deputado/ban...


In [14]:
congressmen.to_sql('deputados',con,schema='viz_painel_temas',if_exists='replace')

### **Political Parties**

#### Get political parties full names

From *camdep_partidos* table.

In [15]:
query_parties = "SELECT DISTINCT idpartido, nomepartido FROM c_camdep.camdep_partidos"
parties = pd.read_sql_query(query_parties,con).set_index('idpartido')
parties.to_sql('partidos', con, schema='viz_painel_temas', if_exists='replace')

---

## **TABLES' FILTERS**

### Propositions table's filter

In [16]:
def propositions_filter(themes_list, years_range):
    """
    Return a json generated by the propositions table filtered by the params.
    :param themes_list: list - Themes of propositions
    :param years_range: list - Years range of propositions
    :return: str(json)
    """
    print('### FILTERING PROPOSITIONS ###')

    if len(themes_list) > 0:
        st = '({}) AND'
        sql_theme_filter = st.format(" OR ".join(["cod_themes LIKE '%%{}%%'".format(theme) for theme in themes_list]))
    else:
        sql_theme_filter = ''

    query = """
    SELECT
        *
    FROM
        viz_painel_temas.proposicoes
    WHERE
        {themes}
        (
          EXTRACT(YEAR FROM dataapresentacao) <= {y1}
          AND (
            descricao_situacao = 'Em Tramitação'
            OR (
              descricao_situacao != 'Em Tramitação'
              AND EXTRACT(YEAR FROM data_ultima_tramitacao) >= {y0}
            )
          )
        )
    """.format(themes=sql_theme_filter, y0=years_range[0], y1=years_range[1])

    df = pd.read_sql_query(query, con)
    df_json = df.to_json(date_format='iso', orient='split')

    print('### DONE ###')
    return df_json

In [17]:
years_range = [2014,2018]
test = propositions_filter(['saude'],years_range)
print(type(test))
test2 = pd.read_json(test,orient='split')
test2.head(2)

### FILTERING PROPOSITIONS ###


ProgrammingError: (psycopg2.ProgrammingError) column "cod_themes" does not exist
LINE 7:         (cod_themes LIKE '%saude%') AND
                 ^
 [SQL: "\n    SELECT\n        *\n    FROM\n        viz_painel_temas.proposicoes\n    WHERE\n        (cod_themes LIKE '%%saude%%') AND\n        (\n          EXTRACT(YEAR FROM dataapresentacao) <= 2018\n          AND (\n            descricao_situacao = 'Em Tramitação'\n            OR (\n              descricao_situacao != 'Em Tramitação'\n              AND EXTRACT(YEAR FROM data_ultima_tramitacao) >= 2014\n            )\n          )\n        )\n    "]

### Congressmen table's filter

In [11]:
def congressmen_filter(propositions_table, years_range):
    """
    Return a json generated by the congressmen table filtered by the params.
    :param propositions_table: str - json with the filtered propositions.
    :param years_range: list - Years range of propositions
    :return: str(json)
    """
    print('### FILTERING CONGRESSMEN ###')

    propositions = pd.read_json(propositions_table, orient='split')
    years_legs = {year: int((year - 2003) / 4 + 52) for year in range(1980, 2030)}

    query = """
    SELECT
        *
    FROM
        viz_painel_temas.deputados
    WHERE
        numlegislatura BETWEEN {leg0} AND {leg1}
    ORDER BY
        numlegislatura DESC;
    """.format(leg0=years_legs[years_range[0]], leg1=years_legs[years_range[1]])

    df = pd.read_sql_query(query, con)

    propositions.idecadastro = np.nan_to_num(propositions.idecadastro).astype(int)
    sponsors = pd.DataFrame(propositions.idecadastro.value_counts())
    sponsors.columns = ['autorias']

    df = df.merge(sponsors, left_on='idecadastro', right_index=True, how='left')
    df_json = df.to_json(date_format='iso', orient='split')

    print('### DONE ###')
    return df_json

In [12]:
test_congressmen = congressmen_filter(test,years_range)
test_congressmen2 = pd.read_json(test_congressmen,orient='split')
test_congressmen2.head(2)

### FILTERING CONGRESSMEN ###
### DONE ###


,index,idecadastro,numlegislatura,nomeparlamentar,sexo,legendapartidoeleito,condicao,situacaomandato,ufeleito,urlfoto,autorias
0,3092,74277,55,LOBBE NETO,M,PSDB,Suplente,Em Exercício,SP,http://www.camara.leg.br/internet/deputado/ban...,4.0
1,882,74651,55,CARLOS MELLES,M,DEM,Titular,Em Exercício,MG,http://www.camara.leg.br/internet/deputado/ban...,NaN


In [13]:
len(test_congressmen2)

1216

### Political parties table's filter

In [16]:
def political_parties_filter(congressmen_table):
    """
    Return a json generated by the political parties table with amount of congressmen and sponsors.
    :param congressmen_table: str - json with the filtered congressmen.
    :return: str(json)
    """
    print('### FILTERING POLITICAL PARTIES ###')
    congressmen = pd\
        .read_json(congressmen_table, orient='split') \
        .rename(columns={'legendapartidoeleito': 'idpartido'})\
        .sort_values('numlegislatura', ascending=False)\
        .drop_duplicates(['idecadastro', 'idpartido'], keep='first')

    names = pd.read_sql_query("SELECT * FROM viz_painel_temas.partidos", con)

    sponsors = congressmen[['idpartido', 'autorias']] \
        .dropna() \
        .groupby(['idpartido']) \
        .agg({'idpartido': 'count', 'autorias': 'sum'}) \
        .rename(columns={'idpartido': 'autores'})

    df = sponsors.merge(names, left_index=True, right_on='idpartido', how='left') \
        .sort_values('autorias', ascending=False)

    df['autoriasporautores'] = df['autorias']/df['autores']

    df_json = df.to_json(date_format='iso', orient='split')

    print('### DONE ###')
    return df_json

In [17]:
test_parties = political_parties_filter(test_congressmen)
test_parties2 = pd.read_json(test_parties,orient='split')
test_parties2.head(2)

### FILTERING POLITICAL PARTIES ###
### DONE ###


,autorias,autores,idpartido,nomepartido,autoriasporautores
76,221,74,PMDB,Partido do Movimento Democrático Brasileiro,2.986486
28,207,57,PSDB,Partido da Social Democracia Brasileira,3.631579


In [62]:
propositions = test_parties2.loc[:,['idpartido','autorias']].sort_values('autorias',ascending=0)
propositions.head(2)

,idpartido,autorias
76,PMDB,221
28,PSDB,207


In [68]:
test_parties2 = test_parties2.set_index('idpartido')

In [153]:
def parties_infos(df_json):
    politcal_parties = pd.read_json(df_json, orient='split')
    fig = plotly.tools.make_subplots(
        rows=1, 
        cols=3, 
        subplot_titles=['Proposições apresentadas', 'Número de autores', 'Relação apresentações por autores'],
        print_grid=False
    )

    fig['layout'].update(
        height = 700,
        showlegend = False,
    )

    trace1 = go.Bar(
        x=political_parties.autorias.sort_values().values,
        y=political_parties.autorias.sort_values().index,
        orientation = 'h',
        marker = dict(
            color = '#00B3E0'
        ),
    )

    trace2 = go.Bar(
        x=political_parties.autores.sort_values().values,
        y=political_parties.autores.sort_values().index,
        orientation = 'h',
        marker = dict(
            color = '#00B3E0'
        ),
    )

    trace3 = go.Bar(
        x=political_parties.autoriasporautores.sort_values().values,
        y=political_parties.autoriasporautores.sort_values().index,
        orientation = 'h',
        marker = dict(
            color = '#00B3E0'
        ),
    )

    fig['layout'].update(
        yaxis=dict(ticksuffix='  '),
        yaxis2=dict(ticksuffix='  '),
        yaxis3=dict(ticksuffix='  '),
    )

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 2)
    fig.append_trace(trace3, 1, 3)

    return dcc.Graph(id='political_parties_graph', figure=fig)

SyntaxError: invalid syntax (<ipython-input-153-070855f0db2e>, line 1)

In [118]:
df = test_parties2.sort_values('autorias')

data = [
    go.Parcoords(
        domain = dict(
            y = df.index
        ),
#        showlegend=True,
#        line = dict(color = df['species_id'],
#                   colorscale = [[0,'#D7C16B'],[0.5,'#23D8C3'],[1,'#F3F10F']]),
        dimensions = list([
            dict(
                ticktext=df.index,
                tickvals=df.autorias,
                label = 'Partidos',
                values = df.autorias
            ),
            dict(
                label = 'Qtd Apresentações',
                values = df.autorias
            ),
            dict(
                label = 'Qtd Autores',
                values = df.autores
            ),
            dict(
                label = 'Relação apresentações por autores',
                values = df.autoriasporautores
            ),
        ])
    )
]

layout = go.Layout(
    height = 1000
)

fig = go.Figure(data = data, layout = layout)
offline.iplot(fig, filename = 'parcoords-basic')

In [110]:
df.index

Index(['PSDC', 'PSL', 'PEN', 'PTN', 'PRP', 'PTdoB', 'PTC', 'PRTB', 'PMN',
       'PSOL', 'PROS', 'PHS', 'PPS', 'SD', 'PCdoB', 'PSC', 'PTB', 'PV', 'PSB',
       'PDT', 'DEM', 'PSD', 'PR', 'PP', 'PRB', 'PT', 'PSDB', 'PMDB'],
      dtype='object', name='idpartido')

In [91]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1


In [56]:
data = [go.Bar(
            x=propositions.autorias,
            y=propositions.idpartido,
            orientation = 'h'
)]

layout = go.Layout(
    yaxis=dict(
        autotick=False,
    ),
    height = 700,
    width = 350
)

fig = go.Figure(data=data, layout=layout)

offline.iplot(fig)

---

## **Infos generators**

In [ ]:
def political_parties_ranking(df_json):
    df = pd.read_json(congressmen_json,orient='split')
    
    ranking = [
        html.Div("""
            {abb}
            {name}
            {sponsors} Autorias - {congressmen} Parlamentares
        """.format(
            abb= df['idpartido'][index],
            name= df['nomepartido'][index],
            sponsors= df['autorias'][index],
            congressmen= df['parlamentares'][index]
        ))
        for index in df[:5].index]

In [45]:
def congressmen_ranking(congressmen_json):
    df = pd.read_json(congressmen_json,orient='split')
    df = df.drop_duplicates('idecadastro')

    ranking = [
        html.Div("""
            {url_picture}
            {name}
            {party} - {district} - {proprositions} Autorias
        """.format(
            url_picture= df['urlfoto'][index],
            name= df['nomeparlamentar'][index],
            party= df['legendapartidoeleito'][index],
            district= df['ufeleito'][index],
            proprositions= df['autorias'][index]
        ))
        for index in df[:3].index]
    
    return ranking

In [ ]:
def propositions_table(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df.ix[:,[
        'idproposicao','tema','nomeproposicao','autor','dataapresentacao',
        'ultimodespacho','data_ultima_tramitacao','situacao'
    ]]
    df.columns = [
        'ID','Temas','Proposição','Autor','Data de Apresentação',
        'Último despacho','Data da última Tramitação','Situação atual'
    ]
    
    return generate_table(df)

In [100]:
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )

In [8]:
def up_pec_disaggregated(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PEC ')]
    
    approved = len(df[df.descricao_situacao == 'Aprovada'])
    processing = len(df[df.descricao_situacao == 'Em tramitação'])
    archived = len(df[df.descricao_situacao == 'Arquivada'])
    
    div = html.Div([
        html.Div("{} Aprovadas".format(approved)),
        html.Div("{} Em tramitação".format(processing)),
        html.Div("{} Arquivadas".format(archived))
    ])
    
    return div

def up_pl_disaggregated(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PL ')]
    
    approved = len(df[df.descricao_situacao == 'Aprovada'])
    processing = len(df[df.descricao_situacao == 'Em tramitação'])
    archived = len(df[df.descricao_situacao == 'Arquivada'])
    
    div = html.Div([
        html.Div("{} Aprovadas".format(approved)),
        html.Div("{} Em tramitação".format(processing)),
        html.Div("{} Arquivadas".format(archived))
    ])
    
    return div

def up_mpv_disaggregated(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('MPV ')]
    
    approved = len(df[df.descricao_situacao == 'Aprovada'])
    processing = len(df[df.descricao_situacao == 'Em tramitação'])
    archived = len(df[df.descricao_situacao == 'Arquivada'])
    
    div = html.Div([
        html.Div("{} Aprovadas".format(approved)),
        html.Div("{} Em tramitação".format(processing)),
        html.Div("{} Arquivadas".format(archived))
    ])
    
    return div

def up_plp_disaggregated(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PLP ')]
    
    approved = len(df[df.descricao_situacao == 'Aprovada'])
    processing = len(df[df.descricao_situacao == 'Em tramitação'])
    archived = len(df[df.descricao_situacao == 'Arquivada'])
    
    div = html.Div([
        html.Div("{} Aprovadas".format(approved)),
        html.Div("{} Em tramitação".format(processing)),
        html.Div("{} Arquivadas".format(archived))
    ])
    
    return div

def up_plv_disaggregated(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PLV ')]
    
    approved = len(df[df.descricao_situacao == 'Aprovada'])
    processing = len(df[df.descricao_situacao == 'Em tramitação'])
    archived = len(df[df.descricao_situacao == 'Arquivada'])
    
    div = html.Div([
        html.Div("{} Aprovadas".format(approved)),
        html.Div("{} Em tramitação".format(processing)),
        html.Div("{} Arquivadas".format(archived))
    ])
    
    return div

def up_pdc_disaggregated(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PDC ')]
    
    approved = len(df[df.descricao_situacao == 'Aprovada'])
    processing = len(df[df.descricao_situacao == 'Em tramitação'])
    archived = len(df[df.descricao_situacao == 'Arquivada'])
    
    div = html.Div([
        html.Div("{} Aprovadas".format(approved)),
        html.Div("{} Em tramitação".format(processing)),
        html.Div("{} Arquivadas".format(archived))
    ])
    
    return div

In [7]:
def up_pec_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PEC ')]
    return len(df)

def up_pl_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PL ')]
    return len(df)

def up_mpv_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('MPV ')]
    return len(df)

def up_plp_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PLP ')]
    return len(df)

def up_plv_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PLV ')]
    return len(df)

def up_pdc_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.nomeproposicao.str.startswith('PDC ')]
    return len(df)

In [6]:
def up_propositions_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    return len(df)

def up_approved_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.descricao_situacao == 'Aprovada']
    return len(df)

def up_processing_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.descricao_situacao == 'Em tramitação']
    return len(df)

def up_archived_amount(df_json):
    df = pd.read_json(df_json,orient='split')
    df = df[df.descricao_situacao == 'Arquivada']
    return len(df)